# A Post Office – A Real Life Story

*Paul Bayer, 2017-07-24*

Let us begin with an everyday story: there is a small post office with one clerk serving the arriving customers. Customers have differing wishes leading to different serving times, from `1 - 5 minutes`. We have to add a little variation to serving times counting for variation in customer habits and clerk performance. The arrival rate of customers is about 18 per hour, every `3.33 minutes` or `3 minutes, 20 seconds` on average. Our post office is small and customer patience is limited, so queue length is limited to 5 customers. 

We have provided 10% extra capacity, so our expectation is that there should not be too many customers discouraged for long waiting times or for full queues.

![](pictures/PostOffice.png)

Let's do a simulation in [```Julia```](https://julialang.org) using [```SimJulia```](https://github.com/BenLauwens/SimJulia.jl). We need 

1. a source: all the **people**, providing an unlimited supply for customers,
2. **customers** with their demands and their limited patience,
3. a **queue** and
4. our good old **clerk**.

In [1]:
using SimJulia, Distributions

function people(sim::Simulation, β::Float64, queue::Array{Int64,1}, clerk::Resource)
    i = 1
    while true
        Δt = rand(Exponential(β))
        yield(Timeout(sim, Δt))
        @process customer(sim, i, queue, clerk)
        i += 1
    end
end

function customer(sim::Simulation, n::Int64, queue::Array{Int64,1}, clerk::Resource)
    if length(queue) ≥ 5
        println(@sprintf("%0.2f: Queue is full! Customer %d leaves", now(sim), n))
        return
    else
        arrivaltime = now(sim)
        println(@sprintf("%0.2f: Customer %d enqueues", now(sim), n))
        push!(queue, n)
        yield(Request(clerk))
        shift!(queue)
        println(@sprintf("%0.2f: Customer %d being served", now(sim), n))
        Δt = rand(DiscreteUniform(1, 5)) + randn()*0.2
        yield(Timeout(sim, Δt))
        yield(Release(clerk))
        println(@sprintf("%0.2f: Customer %d leaves after %0.2f min", now(sim), n, now(sim)-arrivaltime))
    end
end

srand(1234)       # seed random number generator for reproducibility
queue = Int64[]
sim = Simulation()
clerk = Resource(sim, 1)
@process people(sim, 3.333, queue, clerk)
run(sim, 600)
println(queue, " yet in queue")


8.28: Customer 1 enqueues
8.28: Customer 1 being served
13.10: Customer 1 leaves after 4.82 min
13.33: Customer 2 enqueues
13.33: Customer 2 being served
17.44: Customer 2 leaves after 4.11 min
17.69: Customer 3 enqueues
17.69: Customer 3 being served
18.84: Customer 4 enqueues
19.58: Customer 3 leaves after 1.90 min
19.58: Customer 4 being served
21.10: Customer 5 enqueues
21.61: Customer 4 leaves after 2.77 min
21.61: Customer 5 being served
24.63: Customer 5 leaves after 3.53 min
25.66: Customer 6 enqueues
25.66: Customer 6 being served
26.72: Customer 7 enqueues
27.68: Customer 6 leaves after 2.01 min
27.68: Customer 7 being served
28.40: Customer 7 leaves after 1.67 min
29.60: Customer 8 enqueues
29.60: Customer 8 being served
33.96: Customer 8 leaves after 4.36 min
35.70: Customer 9 enqueues
35.70: Customer 9 being served
35.94: Customer 10 enqueues
37.13: Customer 11 enqueues
37.73: Customer 12 enqueues
37.85: Customer 9 leaves after 2.15 min
37.85: Customer 10 being served
40.9

346.74: Customer 104 being served
347.94: Customer 104 leaves after 13.38 min
347.94: Customer 105 being served
350.65: Customer 112 enqueues
352.79: Customer 105 leaves after 18.16 min
352.79: Customer 106 being served
353.80: Customer 106 leaves after 16.79 min
353.80: Customer 108 being served
357.74: Customer 113 enqueues
357.78: Customer 114 enqueues
358.77: Customer 108 leaves after 17.90 min
358.77: Customer 110 being served
361.83: Customer 110 leaves after 19.53 min
361.83: Customer 112 being served
364.21: Customer 115 enqueues
364.58: Customer 116 enqueues
364.99: Customer 117 enqueues
365.80: Customer 112 leaves after 15.15 min
365.80: Customer 113 being served
367.61: Customer 118 enqueues
370.19: Queue is full! Customer 119 leaves
370.52: Customer 113 leaves after 12.78 min
370.52: Customer 114 being served
372.65: Customer 120 enqueues
373.43: Queue is full! Customer 121 leaves
373.86: Customer 114 leaves after 16.08 min
373.86: Customer 115 being served
375.22: Customer

We did run the simulation `600` minutes. `188` customers came, `159` customers were served, `19` left beforehand because the queue was full. Many customers had waiting times of more than 10, 15 up to even 20 minutes.

So many customers will remain angry. If this is the situation all days, our post office will have an evil reputation. What should we do?

## Conclusion

What we see here are the **effects of variation** in arrivals, in demands and in serving time on system performance. In this case 10% extra capacity is not enough to provide for variation and to serve the customers well.

If we had known the situation beforehand we could have provided stand-by for our clerk or install an automatic stamp dispenser for cutting the short tasks … 

Even for such a simple everyday system we cannot say beforehand – without reality check – which throughput, waiting times, mean queue length, capacity utilization or customer satisfaction will emerge. Even more so for more complicated systems in production, service, projects and supply chains with multiple dependencies.

This is what this Repository `PFlow` is all about: to **check assumptions by means of simulation** and to see what may happen beforehand and to get processes and systems flow before they fail to deliver their expected outcomes. It is clear that it must be **easier** to do than to write programs in Julia and to read and interpret long lists as demonstrated. The objective of this repository is to provide a simple framework for such tasks.